<a href="https://colab.research.google.com/github/ptran1203/traffic_sign_detection/blob/main/traffic_signs_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive, output

# drive.mount('/content/drive')
 
%cd "/content"
!rm -rf traffic_sign_detection
!git clone https://github.com/ptran1203/traffic_sign_detection
%cd traffic_sign_detection
output.clear()

In [2]:
!./download_data.sh

Requirement already up-to-date: kaggle in /usr/local/lib/python3.7/dist-packages (1.5.12)


In [ ]:
!python3 train.py --input /content/dataset/za_traffic_2020/traffic_train/images\
                  --backbone densenet121\
                  --batch-size 4\
                  --checkpoint-dir /content/weights\
                  --init-from weights/weight_densenet121.h5\

In [ ]:
!python3 prediction.py --input images/test_image.jpeg\
                       --output /content/submission.json\
                       --weight /content/weights/weight_densenet121.h5\
                    #    --weight weights/weight_densenet121.h5\

In [5]:
!rm /content/infernece_images/*